In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
pre = 'https://en.wiktionary.org/wiki/User:Matthias_Buchmeier/en-fr-'
list_alphabet = [str(chr(one)) for one in range(97,123)]
list_alphabet.insert(len(list_alphabet), '0')

In [ ]:
def find_pos(s):
  x = ''
  try:   
    x = s.split('{')[1].split('}')[0]
  except:
    x = ''
  return x
def find_see_after(s):
  x = ''
  try:   
    x = s.split('SEE:')[1].strip()
  except:
    x = ''
  return x
def find_sens(s):
  x = ''
  try:   
    x = s.split('(')[1].split(')')[0].strip()
  except:
    x = ''
  return x
def to_line_dict(source, sens, see_after, pos):
  line = '{}#{}#{}#{}'.format(source,sens, see_after, pos)
  return line

In [ ]:
to_line_dict('a','b','d','e')

'a#b#d#e'

In [ ]:
dict_pos = {
    "prefix":"Prefix",
    "numeral":"numeral",
    "phraseologicalUnit":"phraseologicalUnit",
    "determiner":"determiner",
    "postposition":"postposition",
    "adj":"adjective",
    "v":"verb",
    "pronoun":"Pronoun",
    "interjection":"interjection",
    "conjunction":"conjunction",
    "n":"noun",
    "pn":"pronoun",
    "suffix":"Suffix",
    "proverb":"proverb",
    "adv":"adverb",
    "article":"Article",
    "symbol":"Symbol",
    "particle":"particle",
    "preposition":"preposition",
}

In [ ]:
find_after = []
all = []
list_pos = list(dict_pos.keys())
for character in list_alphabet:
  r = requests.get(pre+character)
  soup = BeautifulSoup(r.content)
  list_tr = soup.find_all('tr')
  for tr_tag in list_tr:
    source = ''
    pos = ''
    sens = ''
    targets = []
    try:
      source = tr_tag.td.a.text
      source_sense = tr_tag.td.text
      pos = find_pos(source_sense)
      sens = find_sens(source_sense)
      if pos in list_pos:
        pos = dict_pos[pos]
      if 'SEE' in source_sense:
        see_after = find_see_after(source_sense)
        find_after.append(to_line_dict(source,sens,see_after,pos))
        continue
      len_targets = len(tr_tag.find_all('td')[1].find_all('a'))
      for index in range(len_targets):
        a_text = tr_tag.find_all('td')[1].find_all('a')[index].text
        targets.append(to_line_dict(source,sens,a_text,pos))
      for x in targets:
        all.append(x)
    except:
      continue

In [ ]:
print(len(find_after))
list.sort(find_after)

8275


In [ ]:
for i in range(len(find_after)-1):
  if find_after[i]==find_after[i+1]:
    print(i)
    break

5217


In [ ]:
find_after[5215:5220]

['motorbiker#motorcyclist#motorcyclist#noun',
 'motorcar#enclosed passenger vehicle powered by engine#automobile#noun',
 'mould#mould#mold#noun',
 'mould#mould#mold#noun',
 'mould#mould#mold#verb']

In [ ]:
s = '{prop} (Commonwealth of Australia) SEE: Commonwealth of Australia'
print(s.split('SEE:')[1].strip())
s.split('(')[1].split(')')[0].strip()

Commonwealth of Australia


'Commonwealth of Australia'

In [ ]:
import pandas as pd
from io import StringIO

In [119]:
find_after.insert(0,'word#sens#see#pos')
text_find_after = '\n'.join(find_after)
TESTDATA = StringIO(text_find_after)
df1 = pd.read_csv(TESTDATA, error_bad_lines=False, delimiter='#')
df1.head(8)

b'Skipping line 2879: expected 4 fields, saw 5\nSkipping line 4109: expected 4 fields, saw 5\nSkipping line 6066: expected 4 fields, saw 5\nSkipping line 6067: expected 4 fields, saw 5\n'


,word,sens,see,pos
0,-'s,-s,-s,Suffix
1,-form,shaped,shaped,Suffix
2,-in',-ing,-ing,Suffix
3,-ise,-ize,-ize,Suffix
4,-ista,follower,-ist,Suffix
5,-landia,"territory, country or region",-land,Suffix
6,-logic,-logical,-logical,Suffix
7,-san,Miss,Miss,Suffix


In [120]:
df1  = df1.rename(columns={"word": "word_origin"})
df1

,word_origin,sens,see,pos
0,-'s,-s,-s,Suffix
1,-form,shaped,shaped,Suffix
2,-in',-ing,-ing,Suffix
3,-ise,-ize,-ize,Suffix
4,-ista,follower,-ist,Suffix
...,...,...,...,...
8266,zui quan,Drunken Fist,Drunken Fist,noun
8267,zygoma,cheekbone,cheekbone,noun
8268,zygomatic bone,cheekbone,cheekbone,noun
8269,Åland,Åland,Aland,prop


In [114]:
all.insert(0,'word#sens#trans#pos')
text_all = '\n'.join(all)
ALL_DATA = StringIO(text_all)
df2 = pd.read_csv(ALL_DATA, error_bad_lines=False,delimiter='#')

b'Skipping line 41734: expected 4 fields, saw 5\nSkipping line 41735: expected 4 fields, saw 5\nSkipping line 77995: expected 4 fields, saw 5\nSkipping line 77996: expected 4 fields, saw 5\n'


In [115]:
df2

,word,sens,trans,pos
0,a-,"not, without, opposite of",a-,Prefix
1,A,playing card,A,noun
2,AA,Alcoholics Anonymous,AA,prop
3,Aachen,city in North Rhine-Westphalia,Aix-la-Chapelle,prop
4,Aalenian,subdivision of the Middle Jurassic epoch,Aalénien,prop
...,...,...,...,...
100443,-'s,the house/place/establishment of,chez,particle
100444,'sblood,exclamation indicating surprise or emphasis,palsambleu,interj
100445,'sblood,exclamation indicating surprise or emphasis,parsambleu,interj
100446,'s-Hertogenbosch,capital city of North Brabant,Bois-le-Duc,prop


In [121]:
x = df1.set_index('see').join(df2.set_index('word'),lsuffix='_left', rsuffix='_right')
x

,word_origin,sens_left,pos_left,sens_right,trans,pos_right
-ing,-in',-ing,Suffix,to form gerunds,-age,Suffix
-ing,-in',-ing,Suffix,act of doing something,-age,Suffix
-ing,-in',-ing,Suffix,act of doing something,-tion,Suffix
-ing,-in',-ing,Suffix,act of doing something,-ment,Suffix
-ing,-in',-ing,Suffix,to form present participles,-ant,Suffix
...,...,...,...,...,...,...
zipline,zip-wire,zipline,noun,pulley suspended on a cable,tyrolienne,noun
zodiac,girdle,zodiac,noun,belt-like region in the sky,zodiaque,noun
zodiac,girdle,zodiac,noun,twelve signs in astrology,zodiaque,noun
zoological,zoologic,zoological,adjective,"of, or relating to zoology",zoologique,adjective


In [122]:
x.loc[x['word_origin']=='abecedarium']

,word_origin,sens_left,pos_left,sens_right,trans,pos_right
primer,abecedarium,a book used to teach the alphabet,noun,children's book introducing basic literacy,abécédaire,noun
primer,abecedarium,a book used to teach the alphabet,noun,introductory textbook,abc,noun
primer,abecedarium,a book used to teach the alphabet,noun,introductory textbook,abécédaire,noun
primer,abecedarium,a book used to teach the alphabet,noun,substance used to ignite a fire,amorce,noun
primer,abecedarium,a book used to teach the alphabet,noun,small charge used to ignite gunpowder or other...,amorce,noun
primer,abecedarium,a book used to teach the alphabet,noun,layer of priming substance,sous-couche,noun
primer,abecedarium,a book used to teach the alphabet,noun,biology: molecule required for replication of DNA,amorce,noun


In [124]:
df_check = x.reset_index()[['word_origin','sens_left','trans','pos_left']].rename(columns={"word_origin": "word","sens_left": "sens","pos_left": "pos"})
df_check = df_check.drop_duplicates().reset_index().drop(columns='index')
df_check

,word,sens,trans,pos
0,-in',-ing,-age,Suffix
1,-in',-ing,-tion,Suffix
2,-in',-ing,-ment,Suffix
3,-in',-ing,-ant,Suffix
4,-ista,follower,-iste,Suffix
...,...,...,...,...
18261,zipper,zip fastener,tirette,noun
18262,zip-wire,zipline,tyrolienne,noun
18263,girdle,zodiac,zodiaque,noun
18264,zoologic,zoological,zoologique,adjective


In [127]:
df_dic = df2.append(df_check, ignore_index=True).drop_duplicates(ignore_index=True)

In [136]:
df_dic.to_csv('EN-FR-Matthias_Buchmeier_Dic.csv', sep='#',index=False)